In [1]:
import time
import requests
import json
import re
import traceback

import pandas as pd
from bs4 import BeautifulSoup

In [2]:
YANDEX_URL = 'https://yandex.com/indexnow'

In [3]:
PROJECT_SETTINGS_PATH = 'settings.xlsx'

In [4]:
PROJECT_CSV_BASE_PATH = '{}/{}_url_base_indexnow.csv'

## Открывание Settings.xlsx

### лист one map

In [5]:
def get_projects_with_one_map():
    return pd.read_excel(PROJECT_SETTINGS_PATH)

### лист many maps

In [6]:
def get_projects_with_many_maps():
    return pd.read_excel(PROJECT_SETTINGS_PATH, sheet_name = 'many_maps')

## Отчет об ошибках

In [7]:
def error_report():
    print('Ошибка')
    traceback.print_exc()

## Парсинг Sitemap

### Одна карта сайта

In [8]:
def parse_project_with_one_map(project_sitemap):
    try:
        request = requests.get(project_sitemap)
        soup = BeautifulSoup(request.text, 'xml')

        sitemap_url_set = set()
        for url in soup.find_all('loc'):
            sitemap_url_set.add(url.text)

        return sitemap_url_set
    
    except:
        error_report()

### Мультикарта

In [9]:
def parse_project_with_many_maps(project_sitemap):
    try:
        request = requests.get(project_sitemap)

        soup = BeautifulSoup(request.text, 'xml')

        maps = [sitemap.text for sitemap in soup.find_all('loc')]

        print(f'Количество карт: {len(maps)}')

        sitemap_url_set = set()

        i=0
        for sitemap in maps:

            i+=1

            print(f'Карта №: {i}')

            request = requests.get(sitemap)

            print(sitemap, request.status_code)

            soup = BeautifulSoup(request.text, 'xml')

            soup_list = soup.find_all('loc')

            print(f'Количество URL: {len(soup_list)}')

            for url in soup.find_all('loc'):
                sitemap_url_set.add(url.text)

            print(f'Длина общего списка: {len(sitemap_url_set)} URL')

            time.sleep(10)

        return sitemap_url_set
    
    except:
        error_report()

## Актуализация CSV базы

### Открытие csv файла

In [10]:
def open_project_csv_base(project_path, project_name):
    try:
        return pd.read_csv(PROJECT_CSV_BASE_PATH.format(project_path, project_name)).dropna().to_dict('record')
    except:
        error_report()

### Список URL в csv конвертировать в set

In [11]:
def get_url_set_from_project_csv_base(project_csv_base):
    try:
        project_base_url_set = set()
        for row in project_csv_base:
            project_base_url_set.add(row['url'])

        return project_base_url_set
    
    except:
        error_report()

### Список URL в csv конвертировать в list

In [12]:
def get_url_list_from_project_csv_base(project_csv_base):
    try:
        project_base_url_list = []
        for row in project_csv_base:
            project_base_url_list.append(row['url'])

        return project_base_url_list
    
    except:
        error_report()

### Определение новых URL (есть в парсинге Sitemap, нет в CSV)

In [13]:
def find_new_urls_and_set_status_in_csv(project_name, sitemap_url_list_set, project_base_url_list_set):
    try:
        project_csv_base = open_project_csv_base(project_name, project_name)
        
        in_sitemap_not_in_csv_base = sitemap_url_list_set - project_base_url_list_set
        print(f'Количество новых URL (есть в SITEMAP, но нет в CSV базе): {len(in_sitemap_not_in_csv_base)}')
    
        if len(in_sitemap_not_in_csv_base) > 0:
            for element in in_sitemap_not_in_csv_base:
                new_dict = {}
                new_dict['url'] = element
                new_dict['status'] = 'NEW'
                project_csv_base.append(new_dict)
            
            print('Экспорт','\n')
            export_csv_base = pd.DataFrame.from_dict(project_csv_base)        
            export_csv_base.to_csv(PROJECT_CSV_BASE_PATH.format(project_name, project_name), index=False)
            
    except:
        error_report()

### Определение URL без изменений (есть в CSV, есть в Sitemap)

In [14]:
def find_existing_urls_and_set_status_in_csv(project_name, sitemap_url_list_set, project_base_url_list_set):
    try:      
        project_csv_base = open_project_csv_base(project_name, project_name)
        
        in_sitemap_and_in_csv_base = sitemap_url_list_set.intersection(project_base_url_list_set)

        print(f'Количество URL без изменений: {len(in_sitemap_and_in_csv_base)}')

        for row in project_csv_base:
            for url in in_sitemap_and_in_csv_base:
                if row['url'] == url:
                    row['status'] = 'UPDATED'
        
        print('Экспорт','\n')
        export_csv_base = pd.DataFrame.from_dict(project_csv_base)        
        export_csv_base.to_csv(PROJECT_CSV_BASE_PATH.format(project_name, project_name), index=False)
    
    except:
        error_report()

### Определение удаленных URL (есть в CSV, нет в Sitemap)

In [15]:
def find_deleted_urls_and_set_status_in_csv(project_name, sitemap_url_list_set, project_base_url_list_set):
    try:  
        project_csv_base = open_project_csv_base(project_name, project_name)
        
        in_csv_base_not_in_sitemap = project_base_url_list_set - sitemap_url_list_set

        print(f'Количество удаленных с сайта URL (есть в CVS, нет в SITEMAP): {len(in_csv_base_not_in_sitemap)}')

        if len(in_csv_base_not_in_sitemap) > 0:
            for row in project_csv_base:
                for url in in_csv_base_not_in_sitemap:
                    if row['url'] == url:
                        row['status'] = 'DELETED'
            
            print('Экспорт','\n')
            export_csv_base = pd.DataFrame.from_dict(project_csv_base)        
            export_csv_base.to_csv(PROJECT_CSV_BASE_PATH.format(project_name, project_name), index=False)
                        
    except:
        error_report()

### Проверка и сопоставление URL в CSV c данными из Sitemap

In [16]:
def set_status_in_project_csv_base(project_name, sitemap_url_list_set, project_base_url_list_set):
    ### STATUS - NEW
    csv_base_with_new_urls = find_new_urls_and_set_status_in_csv(project_name, 
                                                                 sitemap_url_list_set, 
                                                                 project_base_url_list_set)
            
    
    ### STATUS - UPDATED
    csv_base_with_updated_urls =  find_existing_urls_and_set_status_in_csv(project_name,
                                                                           sitemap_url_list_set, 
                                                                           project_base_url_list_set)
            
    
    ### STATUS - DELETED
    csv_base_with_deleted_urls = find_deleted_urls_and_set_status_in_csv(project_name,
                                                                         sitemap_url_list_set, 
                                                                         project_base_url_list_set)

### Главная функция для актуализации CSV базы для проектов с одной картой

In [17]:
def update_project_csv_base_one_map_main():
    projects_one_map = get_projects_with_one_map()
    projects_main_domain = projects_one_map[projects_one_map['domain_type'] == 'main'].to_dict('record')
    
    for project in projects_main_domain:
        project_name, project_sitemap = project['project'], project['sitemap']
        
        print(project_name)
        
        print('Загрузка CSV базы')
        project_csv_base = open_project_csv_base(project_name, project_name)
        
        print('Конвертация CSV базы в set')
        project_base_url_set = get_url_set_from_project_csv_base(project_csv_base)
        
        print(f'Количество URL в CSV базе: {len(project_base_url_set)}','\n')
        
        print('Парсинг SITEMAP и конвертация в set')
        sitemap_url_set = parse_project_with_one_map(project_sitemap)
        
        print(f'Количество URL в SITEMAP: {len(sitemap_url_set)}','\n')

        print('Сопоставление имеющихся ранее данных с новыми')
        set_status_in_project_csv_base(project_name, sitemap_url_set, project_base_url_set)
        
        print('\n')

### Главная функция для актуализации CSV базы для проектов с мультикартой

In [18]:
def update_project_csv_base_many_maps_main():
    projects_with_many_maps = get_projects_with_many_maps()
    projects_main_domain = projects_with_many_maps[projects_with_many_maps['domain_type'] == 'main'].to_dict('record')
    
    for project in projects_main_domain:
        
        project_name, project_sitemap = project['project'], project['sitemap']
        
        print(project_name)
        
        print('Загрузка CSV базы')
        project_csv_base = open_project_csv_base(project_name, project_name)
        
        print('Конвертация CSV базы в set')
        project_base_url_set = get_url_set_from_project_csv_base(project_csv_base)
        
        print(f'Количество URL в CSV базе: {len(project_base_url_set)}','\n')
        
        print('Парсинг SITEMAP и конвертация в set')
        sitemap_url_set = parse_project_with_many_maps(project_sitemap)
        
        print(f'Количество URL в SITEMAP: {len(sitemap_url_set)}','\n')

        print('Сопоставление имеющихся ранее данных с новыми')
        set_status_in_project_csv_base(project_name, sitemap_url_set, project_base_url_set)
        print('\n')

## Отправка в IndexNow

### Функция, отправляющая не более 10 тыс. URL

In [19]:
def send_urls_index_now(host, key, main_domain, project_name, url_list):
    status_code = 0
    while status_code != 200:
        data = {'host': f'{host}',
                'key': f'{key}',
                'keyLocation': f'https://{main_domain}/{project_name}_private_key_indexnow.txt',
                'urlList': url_list
               }
        
        #Уберите комменты, если нужно посмотреть, какие данные отправляются
        #print(data['host'])
        #print(data['key'])
        #print(data['keyLocation'])
        #print(data['urlList'])

        request = requests.post(YANDEX_URL, json=data)
        
        status_code = request.status_code
        
        if status_code == 202:
            print(f'Ответ {status_code}, ожидание....')
            time.sleep(5)
        
        if status_code == 200:
            print(f'IndexNow - количество отправленных страниц: {len(url_list)},'
                  f' ответ: {status_code}','\n')
            return status_code
            
        if status_code > 202:
            print(f'Ошибка - ответ {status_code}')
            print(f'Причина {request.text}')
            return status_code
            break
            

### Функция, отпраляюшая более 10 тыс. URL

In [20]:
def send_multiple_url_lists_index_now(project_name, project_key, url_list_from_project_csv_base):
    step = 10000
    end = len(url_list_from_project_csv_base)
    start = 0
    total_sent_page_quantity = 0
    check_status_code = 0
    for number in range(start, end, step):
        
        limited_url_list = url_list_from_project_csv_base[number:number+step]
        
        check_status_code = send_urls_index_now(host=project_name, 
                                                key=project_key, 
                                                main_domain=project_name, 
                                                project_name=project_name, 
                                                url_list=limited_url_list)
                            
        if check_status_code > 202:
            return check_status_code
            break
        
        total_sent_page_quantity += len(limited_url_list)
        
        start += step
        
    return check_status_code
        
    print(f'Общее количество отправленных страниц: {total_sent_page_quantity}','\n')

### Получение списка поддоменов

In [21]:
def get_subdomain_list(projects_settings, project_name):
    try:
        projects_subdomain = projects_settings[(projects_settings['domain_type'] == 'subdomain')&
                                              (projects_settings['main_domain_name'] == project_name)].to_dict('record')
    
        return projects_subdomain
    
    except:
        error_report()

### Генерация адресов для поддоменов из основного csv файла

In [22]:
def generate_url_list_for_subdomains(project_domain, project_subdomain, url_list):
    try:
        new_url_list_with_subdomain_name = []
        
        for url in url_list:
            new_url_list_with_subdomain_name.append(re.sub(project_domain, project_subdomain, url))
            
        return new_url_list_with_subdomain_name
    
    except:
        error_report()

### Процесс отправки страниц в IndexNow

In [23]:
def send_urls_indexnow_algorithm(projects_settings, project_name, project_key, project_domain):
    
    print(f'Проект: {project_name}')
    
    project_subdomains_list = get_subdomain_list(projects_settings, project_name)
    
    print(f'Количество доменов: {len(project_subdomains_list)}')
    
    project_csv_base = open_project_csv_base(project_name, project_name)
    
    url_list_from_project_csv_base = get_url_list_from_project_csv_base(project_csv_base)
    
    print(f'Количество URL в базе: {len(url_list_from_project_csv_base)}','\n')
    
    
    if len(url_list_from_project_csv_base) < 10000:
        try:
            check_status_code = send_urls_index_now(host=project_name, 
                                                    key=project_key, 
                                                    main_domain=project_name, 
                                                    project_name=project_name, 
                                                    url_list=url_list_from_project_csv_base)


            if check_status_code > 202:
                print(f'Процесс для проекта {project_name} остановлен')

            if check_status_code <= 202:

                if len(project_subdomains_list) > 0:

                        for row in project_subdomains_list:

                            project_subdomain, subdomain_key = row['project'], row['key']

                            print(f'Поддомен проекта: {project_subdomain}')

                            new_url_list_with_subdomain_name = generate_url_list_for_subdomains(project_domain=project_name, 
                                                                                                project_subdomain=project_subdomain, 
                                                                                                url_list = url_list_from_project_csv_base)

                            send_urls_index_now(host=project_subdomain, 
                                                key=subdomain_key,
                                                main_domain=project_subdomain, 
                                                project_name=project_subdomain, 
                                                url_list=new_url_list_with_subdomain_name)
                            
        except:
            error_report()

                
    if len(url_list_from_project_csv_base) > 10000:
        try:
            check_status_code = send_multiple_url_lists_index_now(project_name, 
                                                                  project_key, 
                                                                  url_list_from_project_csv_base)

            if check_status_code > 202:
                print(f'Процесс для проекта {project_name} остановлен')

            if check_status_code <= 202:

                if len(project_subdomains_list) > 0:
                    
                    for row in project_subdomains_list:
                        
                        project_subdomain, subdomain_key = row['project'], row['key']

                        print(f'Поддомен проекта: {project_subdomain}')

                        new_url_list_with_subdomain_name = generate_url_list_for_subdomains(project_domain=project_name, 
                                                                                            project_subdomain=project_subdomain,
                                                                                            url_list = url_list_from_project_csv_base)

                        send_multiple_url_lists_index_now(project_name,
                                                          project_key,
                                                          new_url_list_with_subdomain_name)
        except:
            error_report()
                    

### Удаление страниц 404 из CSV после завершения отправки в IndexNow

In [24]:
def delete_404_pages_from_base(project_name):
    try:
        project_csv_base = pd.read_csv(PROJECT_CSV_BASE_PATH.format(project_name, project_name))
        quantity404_pages = len(project_csv_base[project_csv_base['status'] == 'DELETED']['url'])
        quantity_new_pages = len(project_csv_base[project_csv_base['status'] == 'NEW']['url'])
        quantity_updated_pages = len(project_csv_base[project_csv_base['status'] == 'UPDATED']['url'])
        print(f'Количество страниц NEW: {quantity_new_pages}')
        print(f'Количество страниц UPDATED: {quantity_updated_pages}')
        print(f'Количество страниц 404: {quantity404_pages}','\n')
        if quantity404_pages > 0:
            project_csv_base.drop(project_csv_base[project_csv_base['status'] == 'DELETED'].index, inplace=True)
            print(f'Удалено: {quantity404_pages}','\n')
            project_csv_base.to_csv(PROJECT_CSV_BASE_PATH.format(project_name, project_name),index=False)
    except:
        error_report()

### Главная функция отправки в IndexNow для проектов с одной картой

In [25]:
def send_project_one_map_to_indexnow_main(): 
    
    projects_one_map = get_projects_with_one_map()
    projects_main_domain = projects_one_map[projects_one_map['domain_type'] == 'main'].to_dict('record')
    
    for project in projects_main_domain:
        project_name, project_key = project['project'], project['key']
        
        send_urls_indexnow_algorithm(projects_settings=projects_one_map, 
                                     project_name=project_name, 
                                     project_key=project_key, 
                                     project_domain=project_name)
       
        delete_404_pages_from_base(project_name)

### Главная функция отправки в IndexNow для проектов с мульти картой

In [26]:
def send_project_many_maps_to_indexnow_main(): 
    
    projects_many_maps = get_projects_with_many_maps()
    projects_main_domain = projects_many_maps[projects_many_maps['domain_type'] == 'main'].to_dict('record')
    
    for project in projects_main_domain:
        project_name, project_key = project['project'], project['key']
        
        send_urls_indexnow_algorithm(projects_settings=projects_many_maps, 
                                     project_name=project_name, 
                                     project_key=project_key, 
                                     project_domain=project_name)
        

        delete_404_pages_from_base(project_name)

## Запуск

### Одна карта

In [27]:
update_project_csv_base_one_map_main()

/Users/andreytryastsin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


big-bears.ru
Загрузка CSV базы
Конвертация CSV базы в set
Количество URL в CSV базе: 115 

Парсинг SITEMAP и конвертация в set
Количество URL в SITEMAP: 115 

Сопоставление имеющихся ранее данных с новыми
Количество новых URL (есть в SITEMAP, но нет в CSV базе): 0
Количество URL без изменений: 115
Экспорт 

Количество удаленных с сайта URL (есть в CVS, нет в SITEMAP): 0


skurala.ru
Загрузка CSV базы
Конвертация CSV базы в set
Количество URL в CSV базе: 8014 

Парсинг SITEMAP и конвертация в set
Количество URL в SITEMAP: 8014 

Сопоставление имеющихся ранее данных с новыми
Количество новых URL (есть в SITEMAP, но нет в CSV базе): 0
Количество URL без изменений: 8014
Экспорт 

Количество удаленных с сайта URL (есть в CVS, нет в SITEMAP): 0


megaposm.com
Загрузка CSV базы
Конвертация CSV базы в set
Количество URL в CSV базе: 0 

Парсинг SITEMAP и конвертация в set
Количество URL в SITEMAP: 842 

Сопоставление имеющихся ранее данных с новыми
Количество новых URL (есть в SITEMAP, но нет в

In [28]:
#send_project_one_map_to_indexnow_main()

### Мультикарта

In [29]:
update_project_csv_base_many_maps_main()

kypishiny.ru
Загрузка CSV базы
Конвертация CSV базы в set
Количество URL в CSV базе: 0 

Парсинг SITEMAP и конвертация в set


/Users/andreytryastsin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


Количество карт: 4
Карта №: 1
https://kypishiny.ru/fx-sitemap/?part=0.1 200
Количество URL: 9904
Длина общего списка: 8735 URL
Карта №: 2
https://kypishiny.ru/fx-sitemap/?part=1 200
Количество URL: 20000
Длина общего списка: 28735 URL
Карта №: 3
https://kypishiny.ru/fx-sitemap/?part=2 200
Количество URL: 20000
Длина общего списка: 48735 URL
Карта №: 4
https://kypishiny.ru/fx-sitemap/?part=3 200
Количество URL: 2280
Длина общего списка: 51015 URL
Количество URL в SITEMAP: 51015 

Сопоставление имеющихся ранее данных с новыми
Количество новых URL (есть в SITEMAP, но нет в CSV базе): 51015
Экспорт 

Количество URL без изменений: 0
Экспорт 

Количество удаленных с сайта URL (есть в CVS, нет в SITEMAP): 0


elitewheels.ru
Загрузка CSV базы
Конвертация CSV базы в set
Количество URL в CSV базе: 0 

Парсинг SITEMAP и конвертация в set
Количество карт: 8
Карта №: 1
https://elitewheels.ru/fx-sitemap?part=0.1 200
Количество URL: 20000
Длина общего списка: 19893 URL
Карта №: 2
https://elitewheels.

In [30]:
#send_project_many_maps_to_indexnow_main()